# Capstone Project 1 - Group ID - 1026

Amar Sheth, Soma, Ankit Nigam

###### Import Packages

In [ ]:
import mysql.connector
from mysql.connector import Error

import pandas as pd
import pandas_profiling

import datetime

import matplotlib.pyplot as plt                                    # Plotting library for Python programming language and it's numerical mathematics extension NumPy
import seaborn as sns                                              # Provides a high level interface for drawing attractive and informative statistical graphics
%matplotlib inline
sns.set()

from subprocess import check_output

from googletrans import Translator

###### Define Variable

In [ ]:
MYSQL_USER = MYSQL_PWD = "student"
MYSQL_HOST = "cpanel.insaid.co"
MYSQL_DB = "Capstone1"

###### Connect to Database function

In [ ]:
def connect_db():
    return mysql.connector.connect(
        host=MYSQL_HOST,
        user=MYSQL_USER,
        passwd=MYSQL_PWD,
        database=MYSQL_DB,
        connect_timeout=1000000,
        buffered=True
    )

###### Execute Query function

In [ ]:
def execute_query(query, connection):
    df = pd.read_sql(query, con = connection)
    return df

In [ ]:
def get_data(query):
    df = pd.DataFrame()
    try:
        db_connection = connect_db()
        cursor = db_connection.cursor(buffered=True)
    
        if db_connection.is_connected():
            df = execute_query(query, db_connection)
    
    except Error as e:
        print("Error while connecting to MySQL", e)
        print("Error code:", e.errno)
        print("Error message:", e.msg)
    finally:
        if (db_connection.is_connected()):
            cursor.close()
            db_connection.close()
            print("MySQL connection is closed")
    
    return df

###### Retreive Event Data

In [ ]:
event_data_query_mp = "SELECT event_id, CAST(device_Id AS CHAR) AS device_id, timestamp, longitude, latitude, city, state FROM events_data where state in ('MadhyaPradesh') " 
df_event_data_mp = get_data(event_data_query_mp)
df_event_data_mp.head()

In [ ]:
event_data_query_other = "SELECT event_id, CAST(device_Id AS CHAR) AS device_id, timestamp, longitude, latitude, city, state FROM events_data where state in ('Chhattisgarh', 'Uttaranchal', 'JammuandKashmir', 'JammuandKashmir', 'Goa', 'Nagaland') " #where state in ('MadhyaPradesh')
df_event_data_other = get_data(event_data_query_other)
df_event_data_other.head()

In [ ]:
df_event_data = pd.concat([df_event_data_mp, df_event_data_other])
df_event_data.head()

###### Retreive Gender Age Data

In [ ]:
gender_age_train_query = "SELECT CAST(device_Id AS CHAR) AS device_id, gender, age, `group` FROM gender_age_train WHERE device_id IN (SELECT DISTINCT device_id from events_data where state in ('MadhyaPradesh', 'Chhattisgarh', 'Uttaranchal', 'JammuandKashmir', 'JammuandKashmir', 'Goa', 'Nagaland')) "
df_gender_age_train = get_data(gender_age_train_query)
df_gender_age_train

###### Retreive Phone Brand Device Data

In [ ]:
phone_brand_device_modal_query = "SELECT CAST(device_Id AS CHAR) AS device_id, phone_brand, device_model FROM phone_brand_device_model WHERE device_id IN (SELECT DISTINCT device_id from events_data where state in ('MadhyaPradesh', 'Chhattisgarh', 'Uttaranchal', 'JammuandKashmir', 'JammuandKashmir', 'Goa', 'Nagaland')) "
df_phone_brand_device_modal = get_data(phone_brand_device_modal_query)
df_phone_brand_device_modal

###### Export to CSV

In [ ]:
export_csv = df_event_data.to_csv (r'events_data_1026.csv', index = None, header=True)
export_csv1 = df_gender_age_train.to_csv (r'gender_age_1026.csv', index = None, header=True)
export_csv2 = df_phone_brand_device_modal.to_csv (r'phone_brand_1026.csv', index = None, header=True)

### Data Profiling

###### Gender Age

In [ ]:
df_gender_age_train['age'].unique()

In [ ]:
df_gender_age_train.info()

In [ ]:
df_gender_age_train.describe()

In [ ]:
df_gender_age_train.isnull().sum()

In [ ]:
filename = "gender_age_train_" + datetime.datetime.now().strftime('%Y-%m-%d_%H:%M:%S') + ".html"
print("FileName - " + filename)

profile = pandas_profiling.ProfileReport(df_gender_age_train)
profile.to_file(outputfile="gender_age_train.html")

In [ ]:
df_by_group = df_gender_age_train.groupby('group')
df_by_group

In [ ]:
df_by_group.describe()

In [ ]:
df_by_group_1 = df_by_group.median()
df_by_group_1.head()

In [ ]:
group_age_dict = dict(tuple(df_by_group))
print (group_age_dict)

In [ ]:
Tot = 12
Col = 2

Rows = Tot // Col 
Rows += Tot % Col

i=0

fig, axs = plt.subplots(Rows, Col, figsize=(20,47)) # adjust the geometry based on your number of columns to plot
for ax,col in zip(axs.flatten(), df_by_group_1.index):
    df = group_age_dict.get(df_by_group_1.index[i])
    sns.countplot(x="age", data=df, ax=ax).set_title(col)
    i=i+1

plt.show()

In [ ]:
df_gender_age_train.groupby(['group'])['group'].count()

In [ ]:
plt.figure(figsize=(15,8))
sns.countplot(x='group', data=df_gender_age_train).set_title('Count plot for group.')

###### Phone Brand

###### Event Data

In [ ]:
df_event_data.info()

In [ ]:
df_event_data.describe()

In [ ]:
df_event_data.isnull().sum()

In [ ]:
cols_to_check = ['longitude', 'latitude']
df_event_data['is_na'] = df_event_data[cols_to_check].isnull().apply(lambda x: all(x), axis=1) 
df_event_data[df_event_data['is_na'] == True]

In [ ]:
df_event_data[df_event_data['is_na'] == True]['device_id'].unique()

In [ ]:
df_event_data[df_event_data['device_id'] == '2350979402202438478']

In [ ]:
df_event_data[df_event_data['device_id'].isin(df_event_data[df_event_data['is_na'] == True]['device_id'].unique())]

In [ ]:
df_event_data.groupby(['state', 'city'])['device_id'].count()

In [ ]:
df_event_data.groupby(['state'])['city'].count()

In [ ]:
nan_missing_lat_long_device_id_list = list(df_event_data[df_event_data['is_na'] == True]['device_id'].unique())

In [ ]:
for id in nan_missing_lat_long_device_id_list:
    device_id = df_event_data[((df_event_data['device_id'] == id) & (df_event_data['is_na'] == False))]['device_id'].unique()
    longitude = df_event_data[((df_event_data['device_id'] == id) & (df_event_data['is_na'] == False))]['longitude'].unique()
    latitude = df_event_data[((df_event_data['device_id'] == id) & (df_event_data['is_na'] == False))]['latitude'].unique()
    print(device_id + " -  " + str(longitude) + " -  " + str(latitude))
    df_event_data.loc[((df_event_data['device_id'] == id) & (df_event_data['is_na'] == True)), ['longitude']] = longitude
    df_event_data.loc[((df_event_data['device_id'] == id) & (df_event_data['is_na'] == True)), ['latitude']] = latitude

In [ ]:
cols_to_check = ['longitude', 'latitude']
df_event_data_copy['is_na'] = df_event_data_copy[cols_to_check].isnull().apply(lambda x: all(x), axis=1) 
df_event_data_copy[df_event_data_copy['is_na'] == True]